In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from IPython.display import Markdown
from IPython.display import display

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
warnings.filterwarnings("ignore")
# pd.set_option('display.max_columns', None)

In [2]:
import json

data = json.load(open("./data/dataset.json", "r"))

data = pd.DataFrame(data)
data.sample(10)

,text,author_id,label,created_at,retweet_count,reply_count,like_count,quote_count,in_reply_to_user_id
210341,News: #COVID19 causes sudden strokes in young ...,741314892,real,2020-04-23T15:52:31.000Z,0,1,0,0,False
177113,so antibiotics do not work. Antibiotics should...,817903682847182848,real,2020-04-13T06:17:18.000Z,0,1,0,0,False
144412,DECA CDC going on today. Seeing some great you...,159487652,real,2020-02-18T19:26:11.000Z,3,0,6,0,True
86642,@weareoversight @joncoopertweets If you track ...,1020399973274390536,real,2020-07-02T11:38:37.000Z,0,0,0,0,False
127813,"As of today (April 29), a total of 6,569 COVID...",702153897272614912,real,2020-04-29T23:54:17.000Z,2,0,3,0,True
241982,"Next, my son comes down stairs from sleeping i...",959535310328737792,real,2020-05-01T22:45:39.000Z,4,1,30,0,False
213439,"DHMs for will be similar to Douglas, Sarpy and...",1083021189587656704,real,2020-04-29T19:41:22.000Z,0,1,1,0,False
136701,Latest Updates May 01\n\n➡️954 new cases and 2...,1190410214954151936,real,2020-05-01T12:55:04.000Z,0,0,1,0,True
75406,@Denverite @DavidASachs FFS Sales Taxes are im...,1228013127725268992,real,2020-07-01T19:49:39.000Z,0,0,0,0,False
166334,As a front line health care provider caring fo...,825800023967408129,real,2020-03-12T01:31:18.000Z,0,0,0,0,True


In [3]:
test_data = data.groupby("label").sample(frac=0.2)
data = data.drop(index=test_data.index)

In [4]:
from xgboost import XGBClassifier
from sentence_transformers import SentenceTransformer

2022-03-31 23:54:17.410640: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-31 23:54:17.410675: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
class Model:
    def __init__(self, params):
        xgb_params= {
            "objective": "binary:logistic",
            "n_estimators": params["n_estimators"],
            "max_depth": params["max_depth"],
            "n_jobs": 7,
            "seed": 1000,
            "verbosity":0,
            'tree_method': "gpu_hist",
            "gpu_id": 0,
            "subsample": params["subsample"],#0.7
            "colsample_bytree": params["colsample_bytree"],
            "learning_rate": params["learning_rate"],
            "lambda": params["lambda"],
            "alpha": params["alpha"],
            "eval_metric":'logloss'
        }
        
        self.emb = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")
        self.xgb = XGBClassifier(**xgb_params)
    
    def fit(self, X_train, y_train):
        X_emb = self.emb.encode(X_train)
        self.xgb.fit(X_emb, y_train)
        return self
    
    def predict(self, X_test):
        X_emb = self.emb.encode(X_test)
        preds = self.xgb.predict(X_emb)
        return preds
    
    def predict_proba(self, X_test):
        X_emb = self.emb.encode(X_test)
        preds = self.xgb.predict_proba(X_emb)
        return preds

In [6]:
params = {'ngram_range': 2, 'max_df': 0.6648896815644944, 'max_features': 1928, 'n_estimators': 605, 'max_depth': 10, 'subsample': 0.7159105584464553, 
          'colsample_bytree': 0.7851446292315494, 'learning_rate': 0.13139478930452317, 'lambda': 1.2081963210286712, 'alpha': 1.0087203749646416}

model = Model(params)

In [7]:
model.fit(data.text.values, data.label.values)

In [8]:
probs = model.predict_proba(test_data.text.values)

In [9]:
probs[:5]

array([[9.6880847e-01, 3.1191526e-02],
       [9.9998558e-01, 1.4417770e-05],
       [9.9998301e-01, 1.7009592e-05],
       [3.1091899e-01, 6.8908101e-01],
       [8.4113413e-01, 1.5886585e-01]], dtype=float32)

In [10]:
y = (test_data.label == "fake")
y.head()

51836     True
194010    True
191185    True
183359    True
190972    True
Name: label, dtype: bool

In [11]:
import metrics as mmetrics

In [12]:
y = y.values.reshape((-1, 1))

perfs = mmetrics.get_performance_metrics(y, probs[:, :1], ["ST-XGB"])
perfs

,TP,TN,FP,FN,Accuracy,Prevalence,Sensitivity,Specificity,PPV,NPV,AUC,F1,Threshold
,,,,,,,,,,,,,
ST-XGB,1164,42142,13,969,0.978,0.048,0.546,1,0.989,0.978,0.96,0.703,0.5
